In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import yaml
import warnings
import sys
from ultralytics import YOLO
from pathlib import Path
warnings.filterwarnings('ignore')

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 환경에 따라 PROJECT_ROOT 설정
if IN_COLAB:
    PROJECT_ROOT = Path("/content/drive/MyDrive/likelion/PCB")
else:
    PROJECT_ROOT = Path.cwd().parent

# sys.path 추가 (import용)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 작업 디렉토리 변경 (파일 접근용) - 중요!
os.chdir(PROJECT_ROOT)

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Current working directory: {os.getcwd()}")

Environment: Colab
PROJECT_ROOT: /content/drive/MyDrive/likelion/PCB
Current working directory: /content/drive/MyDrive/likelion/PCB


In [4]:
!nvidia-smi

Wed Jan 14 09:22:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
from src.utils.config import load_config

# 파라미터 확인
config_path = f"{PROJECT_ROOT}/configs/config.yaml"
config = load_config(config_path)
config

{'model': {'type': 'yolo11',
  'size': 'm',
  'pretrained': False,
  'checkpoint': 'weights/best_yolov11m_imgsz1280.pt'},
 'data': {'data_yaml': 'dataset/roboflow/data.yaml', 'img_size': 1280},
 'training': {'epochs': 30,
  'batch_size': 32,
  'freeze': 10,
  'device': 'cuda',
  'workers': 8,
  'multi_scale': True,
  'mosaic': 0.1,
  'mixup': 0.1,
  'box': 7.5,
  'cls': 0.5,
  'optimizer': 'AdamW',
  'lr0': 0.0002,
  'lrf': 0.01,
  'momentum': 0.937,
  'weight_decay': 0.0005,
  'patience': 10,
  'save_period': -1,
  'save_dir': 'outputs'},
 'augmentation': {'enabled': True,
  'pcb_optimized': True,
  'exclude_classes': []},
 'wandb': {'enabled': True, 'project': 'PCB_Detection', 'suffix': 'Finetune'},
 'experiment': {'suffix': 'Finetune', 'seed': 42}}

In [6]:
from main import run_pipeline
result = run_pipeline()

2026-01-14 09:22:17 - INFO - Starting training...
INFO:PCB_Detection:Starting training...
2026-01-14 09:22:17 - INFO - Config: configs/config.yaml
INFO:PCB_Detection:Config: configs/config.yaml
2026-01-14 09:22:17 - INFO - Project root: /content/drive/MyDrive/likelion/PCB
INFO:PCB_Detection:Project root: /content/drive/MyDrive/likelion/PCB
2026-01-14 09:22:17 - INFO - Working directory: /content/drive/MyDrive/likelion/PCB
INFO:PCB_Detection:Working directory: /content/drive/MyDrive/likelion/PCB
2026-01-14 09:22:17 - INFO - Logs will be saved to: /content/drive/MyDrive/likelion/PCB/logs/yolo11m-Finetune
INFO:PCB_Detection:Logs will be saved to: /content/drive/MyDrive/likelion/PCB/logs/yolo11m-Finetune
2026-01-14 09:22:17 - INFO - Seed fixed to 42
INFO:PCB_Detection:Seed fixed to 42
2026-01-14 09:22:17 - INFO - Using device: cuda
INFO:PCB_Detection:Using device: cuda
2026-01-14 09:22:18 - INFO - GPU: NVIDIA A100-SXM4-80GB
INFO:PCB_Detection:GPU: NVIDIA A100-SXM4-80GB
2026-01-14 09:22:18 

2026-01-14 09:22:22 - INFO - W&B initialized for YOLO: project='PCB_Detection', name='yolo11m-Finetune'
INFO:PCB_Detection:W&B initialized for YOLO: project='PCB_Detection', name='yolo11m-Finetune'
2026-01-14 09:22:22 - INFO - Custom W&B callback registered for YOLO (logging only, no console output)
INFO:PCB_Detection:Custom W&B callback registered for YOLO (logging only, no console output)
2026-01-14 09:22:22 - INFO - Training parameters: {'data': '/content/drive/MyDrive/likelion/PCB/dataset/roboflow/data.yaml', 'epochs': 30, 'batch': 32, 'imgsz': 1280, 'patience': 10, 'device': device(type='cuda'), 'workers': 8, 'save': True, 'save_period': -1, 'project': '/content/drive/MyDrive/likelion/PCB/outputs', 'name': 'yolo11m-Finetune', 'val': True, 'plots': False, 'verbose': False, 'optimizer': 'AdamW', 'lr0': 0.0002, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'multi_scale': True, 'mosaic': 0.1, 'mixup': 0.1, 'box': 7.5, 'cls': 0.5}
INFO:PCB_Detection:Training parameters: {'dat

engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/likelion/PCB/dataset/roboflow/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=weights/best_yolov11m_imgsz1280.pt, momentum=0.937, mosaic=0.0, multi_scale=True, name=yolo11m-Finetune, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=False, pose=1

2026-01-14 09:28:17 - ERROR - Training failed: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/drive/MyDrive/likelion/PCB/src/training/custom_trainer.py", line 26, in __getitem__
    return super().__getitem__(index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/data/base.py", line 376, in __getitem__
    return self.transforms(self.get_image_and_label(index))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytic

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/drive/MyDrive/likelion/PCB/src/training/custom_trainer.py", line 26, in __getitem__
    return super().__getitem__(index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/data/base.py", line 376, in __getitem__
    return self.transforms(self.get_image_and_label(index))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/data/augment.py", line 187, in __ca